In [16]:
%pip install plotly

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/a8/07/72953cf70e3bd3a24cbc3e743e6f8539abe6e3e6d83c3c0c83426eaffd39/plotly-5.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e454214013b441c4575/tenacity-8.2.3-py3-none-any.whl.metadata
   ---------------------------------------- 15.6/15.6 MB 622.1 kB/s eta 0:00:00
Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import ridgeplot as rp
import plotly
import numpy as np
import pandas as pd
import requests
from dataclasses import dataclass
from enum import Enum
import time



ModuleNotFoundError: No module named 'ridgeplot'

In [2]:
# each function is a conversion from the unit to DALY/dollar.
class Units(Enum):
    daly_over_dollar = lambda x: x
    dollar_over_daly = lambda x: 1/x
    in_gd_daly_over_dollar = lambda x: x*0.00335/0.4 # GW's estimate of GiveDirectly's impact is 0.00335 doubling of consumption per dollar (https://docs.google.com/spreadsheets/d/18ROI6dRdKsNfXg5gIyBa1_7eYOjowfbw5n65zkrLnvc/edit#gid=1680005064), one doubling of consumption is about 0.4 DALYs

@dataclass
class Spreadsheet:
    name: str
    url: str
    result_cell_coordinate: str
    units: Units = Units.daly_over_dollar
    result_worksheet: str = ""

    def __hash__(self) -> int:
        return hash(self.name + self.url + self.result_cell_coordinate + self.result_worksheet)

In [3]:
spreadsheets = [
    Spreadsheet(
        "Replacement of Iron supplementation",
        "https://docs.google.com/spreadsheets/d/1342XLGqjpLXV4pHXINz2SrzAora2xhm-99x6slt64Xs/",
        "E72",
        Units.dollar_over_daly,
        "Daggered",
    ),
    Spreadsheet(
        "GiveWell IFA CEA",
        "https://docs.google.com/spreadsheets/d/1_ttwAj4rH9rDhqGeil01hBmmrdP2qjIF2RvVMB5BDT8/edit#gid=0",
        "B96",
        Units.in_gd_daly_over_dollar,
        "Main",
    )
]

In [4]:
class Dagger:
    def __init__(self, spreadsheets, sensitivity=False):
        self.spreadsheets = spreadsheets
        self.sensitivity = sensitivity
        self.runs = {spreadsheet: {'id':"", 'result':None, 'status':"NOT_STARTED"} for spreadsheet in spreadsheets}
        
    def run(self):
        self.start_jobs()
        self.get_all_results()
    
    @staticmethod
    def spreadsheet_to_params(spreadsheet: Spreadsheet, sensitivity: bool):
        return {
            "url": spreadsheet.url,
            "result_worksheet": spreadsheet.result_worksheet,
            "result_cell_coordinate": spreadsheet.result_cell_coordinate,
            "sensitivity": sensitivity,
        }
    
    def start_jobs(self):
        post_url = 'https://usedagger.com/api/spreadsheet/'
        for spreadsheet in self.spreadsheets:
            params = self.spreadsheet_to_params(spreadsheet, self.sensitivity)
            result = requests.post(post_url, json=params)
            self.runs[spreadsheet]["id"] = result.json()['id']
            self.runs[spreadsheet]["status"] = "STARTED"

    @staticmethod
    def get_results(id):
        results = requests.get(f'https://usedagger.com/api/sim/{id}')
        status = results.json()['status']['status']
        if status == "SUCCESS":
            return status, pd.DataFrame(results.json()['output']['simulation_data'])
        return status, None
            
    def get_all_results(self, attempts=20, wait=5):
        for spreadsheet in self.spreadsheets:
            if self.runs[spreadsheet]['status'] == "SUCCESS":
                continue
            print(f"Getting results for {spreadsheet.name}")
            for attempt in range(attempts):
                status, result = self.get_results(self.runs[spreadsheet]['id'])
                self.runs[spreadsheet]['status'] = status
                if status == "SUCCESS":
                    self.runs[spreadsheet]['result'] = result
                    break
                if status == "STARTED":
                    time.sleep(wait)
                    continue
                
            if attempt == attempts - 1:
                print(f"Error: could not get results for {spreadsheet.name} after {attempts} attempts")
                print(f"Error: {status} for {spreadsheet.name}")
                print(f"Error: {result}")
                self.runs[spreadsheet]['status'] = "TOO_LONG"
            print(f"Got results for {spreadsheet.name}, status: {self.runs[spreadsheet]['status']}")

    def get_results_df(self):
        df = pd.concat([
            self.runs[spreadsheet]['result'][[spreadsheet.result_worksheet+'!'+spreadsheet.result_cell_coordinate]].rename(
                columns=lambda x: spreadsheet.name
            ).apply(spreadsheet.units) 
            for spreadsheet in self.spreadsheets
            ], axis="columns")
        return df




In [5]:
run = Dagger(spreadsheets)

In [6]:
run.run()

Getting results for Replacement of Iron supplementation
Got results for Replacement of Iron supplementation, status: SUCCESS
Getting results for GiveWell IFA CEA
Got results for GiveWell IFA CEA, status: SUCCESS


In [7]:
df = run.get_results_df()
df.head()

,Replacement of Iron supplementation,GiveWell IFA CEA
0,0.088624,0.037614
1,0.056406,0.056970
2,0.038378,0.031059
3,0.021959,0.056079
4,0.070284,0.053238


In [8]:
fig = rp.ridgeplot(samples=df.to_numpy().T, labels=df.columns.to_list())


In [9]:
fig

In [10]:
fig.show("json")

In [17]:
import plotly.express as px
import plotly.graph_objects as go



In [12]:
fig = go.Figure()
for column in df.columns:
    fig.add_trace(go.Violin(x=df[column]))


In [13]:
fig.update_traces(points=False, meanline_visible=True, side='positive', orientation='h', width=3)
fig.update()

In [3]:
import pandas as pd

df = pd.read_csv("./Mental Health Intervention BOTECs_carlo_aNxHdYVfEhiW2GqSWcfCNG_806.csv")
df.head()

,"Campaign cost per person reached, USD (per year?) ('Mass Media'!B3)",Proportion of target audience who change behavior ('Mass Media'!B4),"Annual burden of IPV per 100,000 women aged 16-49, DALYs ('Mass Media'!B7)","Proportion of women aged 16-49 who experience IPV in a given year, LMIC ('Mass Media'!B8)","Annual burden of IPV per sufferer, DALYs ('Mass Media'!B9)",Proportion of IPV burden attributable to depression ('Mass Media'!B10),"Annual depression burden of IPV per sufferer, DALYs ('Mass Media'!B11)",Proportion of people reached who are target audience (men who are violent to partners) ('Mass Media'!B12),Cost per target audience member reached ('Mass Media'!B13),Benefit per target audience member reached (depression only) ('Mass Media'!B15),...,Proportion of global population covered by water flouridization ('Suicide Prevention'!B29),"Degree to which lithium supplementation is adopted after a study, as a multiple of the adoption of water flouridization ('Suicide Prevention'!B30)",Counterfactual persistence ('Suicide Prevention'!B31),"Global suicide rate per 100,000 per year ('Suicide Prevention'!B32)",New suicide rate after lithium supplementation ('Suicide Prevention'!B33),"Annual cost to governments of lithium supplementation after a study, USD millions ('Suicide Prevention'!B34)","Total cost in expectation, disocunted for lower impact of government spending, USD millions ('Suicide Prevention'!B35)",Suicides averted per advocacy attempt (study population) ('Suicide Prevention'!B36),Suicides averted per advocacy attempt (global) ('Suicide Prevention'!B37),"Cost-effectiveness of lithium-in-water study, DALYs per $100,000 ('Suicide Prevention'!B38)"
0,9.699162,0.410830,261.858060,0.059400,0.044084,0.513406,0.022633,0.050119,193.523240,0.009298,...,0.054604,0.384757,16.975988,4.507020,3.532391,3462.809554,0.643681,147.087118,1635.365326,7248.044147
1,5.084941,0.168735,320.144892,0.255903,0.012510,0.401871,0.005028,0.215918,23.550318,0.000848,...,0.042786,0.240659,69.452712,4.807881,3.588203,2165.926898,9.924792,198.812092,11825.731086,1984.589161
2,3.990613,0.185360,261.193607,0.304858,0.008568,0.724602,0.006208,0.257224,15.514165,0.001151,...,0.054747,0.291582,28.172486,7.427506,5.397349,2624.237731,21.301785,468.005969,24825.932926,2343.983721
3,3.523428,0.188746,1030.230039,0.053826,0.191399,0.900830,0.172418,0.045416,77.581464,0.032543,...,0.049541,0.128084,47.277974,5.381351,3.651460,1152.751814,0.112241,0.872829,192.978318,6270.416472
4,4.729367,0.232478,676.850648,0.119641,0.056573,0.505582,0.028602,0.100947,46.849821,0.006649,...,0.079199,0.073987,22.229427,6.075150,4.779398,665.879353,0.389555,44.178764,2187.141455,16257.618194


In [13]:
output_columns = list(filter(lambda x: x.startswith("Cost-effectiveness") and "DALYs" in x, list(df.columns)))

In [22]:
fig = go.Figure()
for column in output_columns:
    fig.add_trace(go.Violin(y=df.columns, x=df[column]))


In [23]:
fig.update_traces(points=False, meanline_visible=True, side='positive', orientation='h', width=3)
fig.update()


In [55]:
df2 = df[output_columns].copy()
def clean_name(name):
    n = name.lstrip("Cost-effectiveness of ")
    return n[:n.find(", DALYs")]
df2.rename(columns=clean_name, inplace=True)
df2.head()



,radio campaign,SSB tax advocacy for reducing depression & suicide,advocating for a physical activity campaign for reducing depression & suicide,IPT-G,rnet-delivered psychotherapy,advocating for child poverty grant,pesticide ban advocacy,lithium-in-water study
0,4.804731,35.295624,14.427883,73.848007,404.799019,8.563827,7592.472473,7248.044147
1,3.602195,28.419102,55.709278,31.854674,11.785824,18.735341,2861.877309,1984.589161
2,7.417406,7.923677,14.740893,65.826172,125.472645,7.427421,1141.743498,2343.983721
3,41.947280,5.333575,2.009856,16.444414,9.647864,13.190611,13.307724,6270.416472
4,14.193064,2.344179,21.626300,4.676789,34.327401,1.819640,488.270787,16257.618194


In [56]:
# flatten df2 to a dataframe with columns: value, column_name
df3 = pd.DataFrame()
for column in df2.columns:
    df3 = pd.concat([df3, pd.DataFrame({"value": df2[column], "column_name": column})])
df3.head()

,value,column_name
0,4.804731,radio campaign
1,3.602195,radio campaign
2,7.417406,radio campaign
3,41.947280,radio campaign
4,14.193064,radio campaign


In [62]:
fig = go.Figure()
fig.add_trace(go.Violin(x=df3["column_name"], y=df3["value"]))
fig.update_traces(box_visible=True, meanline_visible=True)
